<a href="https://colab.research.google.com/github/lyla-lee/TIL/blob/master/NLP_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#10장 문서 분류 (Document Classification)

#11-1 나이브 베이즈 분류(Naive Bayes Classifier)

##1.1 직접구현

### Naive Bayes Classifier

In [ ]:
training_set = [['me free lottery', 1],
 ['free get free you', 1],
 ['you free scholarship', 0],
 ['free to contact me', 0],
 ['you won award', 0],
 ['you ticket lottery', 1]]

In [ ]:
if 'free' in training_set[1][0]:
  print(training_set[1][0].count('free'))

2


### 토큰 빈도수 및 문서별 토큰수 계산 (확률 계산을 위한 준비)

![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/98f086c560aa2f66650060277dda4f90e54e30c0)

### 방법 1

In [ ]:
word =[]
for i in range(len(training_set)):
  word.append(training_set[i][0])
  word_list = [i.split() for i in word]
word_list

[['me', 'free', 'lottery'],
 ['free', 'get', 'free', 'you'],
 ['you', 'free', 'scholarship'],
 ['free', 'to', 'contact', 'me'],
 ['you', 'won', 'award'],
 ['you', 'ticket', 'lottery']]

In [ ]:
import numpy as np
set_list = set([i for word in word_list for i in word])
wordfreq1 = defaultdict(lambda : 0)
[wordfreq1[unique_word] for unique_word in set_list]
wordfreq2=wordfreq1.copy()
wordfreq2

defaultdict(<function __main__.<lambda>>,
            {'award': 0,
             'contact': 0,
             'free': 0,
             'get': 0,
             'lottery': 0,
             'me': 0,
             'scholarship': 0,
             'ticket': 0,
             'to': 0,
             'won': 0,
             'you': 0})

In [ ]:
for i in range(len(training_set)):
  for unique_word in set_list:
    if unique_word in training_set[i][0] and training_set[i][1]==1:
      wordfreq1[unique_word] += training_set[i][0].count(unique_word)
      # +=1 씩 더하면, 한문장에 free 두개일때 하나만 카운트 되니깐 수정하기
      spam_wordfreq = wordfreq1

    if unique_word in training_set[i][0] and training_set[i][1]==0:
      wordfreq2[unique_word] +=1
      normal_wordfreq = wordfreq2
spam_wordfreq, normal_wordfreq

(defaultdict(<function __main__.<lambda>>,
             {'award': 0,
              'contact': 0,
              'free': 3,
              'get': 1,
              'lottery': 2,
              'me': 1,
              'scholarship': 0,
              'ticket': 1,
              'to': 0,
              'won': 0,
              'you': 2}),
 defaultdict(<function __main__.<lambda>>,
             {'award': 1,
              'contact': 1,
              'free': 2,
              'get': 0,
              'lottery': 0,
              'me': 1,
              'scholarship': 1,
              'ticket': 0,
              'to': 1,
              'won': 1,
              'you': 2}))

### 방법 2: key값에 두개의 빈도수를 넣는 방법

In [ ]:
from collections import defaultdict

# 범주에 속하는 토큰수 세기 1(스팸), 0(정상))
doccnt0 = 0
doccnt1 = 0

# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])

for doc, label in training_set:
  words = doc.split()
  for word in words:
    wordfreq[word][label] += 1

for key, (cnt0, cnt1) in wordfreq.items():
  doccnt0 += cnt0
  doccnt1 += cnt1

print('doccnt0 : {}'.format(doccnt0))
print('doccnt1 : {}'.format(doccnt1))

doccnt0 : 10
doccnt1 : 10


### Training : 토큰별 조건부 확률 계산 

### 방법1

In [ ]:
k = 0.5

spam_probs = {}
normal_probs ={}
for (key,value) in spam_wordfreq.items():
  spam_probs[key] = (k+ value)/ (sum(spam_wordfreq.values())+ 2*k)
for (key,value) in normal_wordfreq.items():
  normal_probs[key] = (k+ value)/ (sum(normal_wordfreq.values())+ 2*k)


spam_probs, normal_probs

({'award': 0.045454545454545456,
  'contact': 0.045454545454545456,
  'free': 0.3181818181818182,
  'get': 0.13636363636363635,
  'lottery': 0.22727272727272727,
  'me': 0.13636363636363635,
  'scholarship': 0.045454545454545456,
  'ticket': 0.13636363636363635,
  'to': 0.045454545454545456,
  'won': 0.045454545454545456,
  'you': 0.22727272727272727},
 {'award': 0.13636363636363635,
  'contact': 0.13636363636363635,
  'free': 0.22727272727272727,
  'get': 0.045454545454545456,
  'lottery': 0.045454545454545456,
  'me': 0.13636363636363635,
  'scholarship': 0.13636363636363635,
  'ticket': 0.045454545454545456,
  'to': 0.13636363636363635,
  'won': 0.13636363636363635,
  'you': 0.22727272727272727})

### 방법 2

In [ ]:
k = 0.5

wordprobs = defaultdict(lambda : [0, 0])
for key, (cnt0, cnt1) in wordfreq.items():
  wordprobs[key][0] = (cnt0 + k) /(2 * k + doccnt0)
  wordprobs[key][1] = (cnt1 + k) /(2 * k + doccnt1)

wordprobs

### Classify : 신규 텍스트가 주어졌을 때 확률 계산

>



### 방법 1

In [ ]:
import math

doc = "free lottery"
tokens = doc.split()
log_prob1 = log_prob0 = 0.0

for word,prob0 in spam_probs.items():
  if word in tokens:
    log_prob0 += math.log(prob0)
for word,prob1 in normal_probs.items():
  if word in tokens:
    log_prob1 += math.log(prob1)

log_prob0 += math.log(sum(spam_wordfreq.values())/(sum(spam_wordfreq.values())+sum(normal_wordfreq.values())))
log_prob1 += math.log(sum(normal_wordfreq.values())/(sum(normal_wordfreq.values())+sum(spam_wordfreq.values())))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print(prob0)
print(prob1)
# print(prob1)
print("스팸확률 : {}".format( prob0 /(prob0 + prob1) * 100))
print("정상확률 : {}".format( prob1 /(prob0 + prob1) * 100))

0.03615702479338842
0.00516528925619835
스팸확률 : 87.49999999999999
정상확률 : 12.500000000000009


### 방법 2

In [ ]:
import math

doc = "free lottery"
tokens = doc.split()

log_prob1 = log_prob0 = 0.0

for word, (prob0, prob1) in wordprobs.items():
  if word in tokens:
    log_prob0 += math.log(prob0) 
    log_prob1 += math.log(prob1)

log_prob0 += math.log(doccnt0/(doccnt0 + doccnt1))
log_prob1 += math.log(doccnt1/(doccnt0 + doccnt1))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print(prob0)
print(prob1)

print("정상확률 : {}".format( prob0 /(prob0 + prob1) * 100))
print("스팸확률 : {}".format( prob1 /(prob0 + prob1) * 100))

# scikit learn 활용하기

In [1]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names)
print(twenty_train.data[0])
print(twenty_train.target[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [4]:
MultinomialNB?

In [5]:
text_clf

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [6]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',  shuffle=True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_clf = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf__alpha' : (1, 0.1, 0.01, 0.001, 0.0001)
                  }

gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs=-1, verbose=2)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
print('Best score : {}'.format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(list(best_parameters.keys())):
  print("\t{} : {}".format(param_name, best_parameters[param_name]))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
predicted = gs_clf.best_estimator_.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)